In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install ../input/talib-package/talib_binary-0.4.19-cp37-cp37m-manylinux1_x86_64.whl
import talib as ta 

Processing /kaggle/input/talib-package/talib_binary-0.4.19-cp37-cp37m-manylinux1_x86_64.whl


In [3]:
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, MinMaxScaler
import missingno as msno

import statsmodels.api as sm
from pylab import rcParams

from tqdm import tqdm

# import ta
from talib import abstract


import time
import gc
import sys

# sys.path.insert(0, '../input/jpx-local-api')
# from local_api import local_api

In [4]:
data = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
# display(data)
train = data.copy()

# using supplement data as test data
supp_data = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv')

# train_with_supp = pd.concat([train, supp_data]).reset_index(drop=True)
train_with_supp = train.copy()
# train_with_supp


In [5]:
# def time_elapsed(t0):
#     t1 = time.time()
#     print(time.time() - t0)
#     return t1

SecuritiesCode 에 따라 인덱싱하는 것이 시간 소요가 너무 크다.  
미리 Data를 종목마다 쪼개서 넣어 놓는게 좋겠다. - > 리스트 활용

In [6]:
def divideSecurities(df):
    sec_list = []
#     print('Divide securities individually..')
    for code in np.sort(df.SecuritiesCode.unique()):
        sec_list.append(df.loc[df.SecuritiesCode == code, :].reset_index(drop=True))
    return sec_list

sec_list = divideSecurities(train_with_supp)

In [7]:
def add_features_train(input_df, sec_list):
    df_list = []
    for df in sec_list:
        
        # shadows
        df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
        df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']
        

        
        # lagged features
        # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
        # lagged close, target (target 은 정확히 무엇? return인가)
        
        # lagged feature 계산하기 전 결측치 채워넣기
        df = df.fillna(method='ffill')
        

        
        # TA-lib features - RSI, EMA 7-90
        df['RSI'] = ta.RSI(df['Close'])
        df['EMA7'] = ta.EMA(df['Close'], 7)
        df['EMA15'] = ta.EMA(df['Close'], 15)
        df['EMA30'] = ta.EMA(df['Close'], 30)
        df['EMA90'] = ta.EMA(df['Close'], 90)
        

        
#         for indicator in ta.get_function_groups()['Pattern Recognition']:
#             df[str(indicator)] = getattr(ta,str(indicator))(df.Open, df.High, df.Low, df.Close)


        # fill ema features by backward -- 이렇게 채워진 것은 false data 이므로 일단 test 해보고 없애는 것을 검토하자.
        df = df.fillna(method='bfill')

    
        # volatility
        
        df_list.append(df)

        
    df_feature_added = pd.concat(df_list)
    
    return df_feature_added


In [8]:
# add feature to data

df_added = add_features_train(train_with_supp, sec_list)
# df_added.to_csv('train_with_supp_feature_added.csv', index=False)

# load data
# df_added = pd.read_csv('/kaggle/input/train-with-supp-feature-added-v1-all-cdl/train_with_supp_feature_added.csv')

# df_added

In [9]:
def add_features_infer(input_df, close_df): #input df 는 price 데이터
    
    df_list = []
    sec_list = divideSecurities(input_df)
    
    close_list = divideSecurities(close_df)
    
    for i in range(len(sec_list)):
        
        
        close = close_list[i] #.loc[close_df.SecuritiesCode == code, :].fillna(method='ffill')
        df = sec_list[i]
#         # test data의 open, high, low, close 중 nan 있으면 이전 값에서 가져와 채움
        if df.loc[:, ['Open', 'High', 'Low', 'Close']].isna().any().any():
#             display(df)
#             display(close)
# #             print(close.iloc[-2]['Open', 'High', 'Low', 'Close'].values())
            df.loc[:, ['Open', 'High', 'Low', 'Close']] = close.loc[close['Date'] == close.iloc[-1]['Date'], ['Open', 'High', 'Low', 'Close']].values 
        
        

        
        # shadows
        df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
        df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']

        # lagged features
        # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
        # lagged close, target (target 은 정확히 무엇? return인가)
        
        ## Rolling features ##
        
        
        # TA-lib features - RSI, EMA 7-90
        df['RSI'] = ta.RSI(close['Close']).iloc[-1]
        df['EMA7'] = ta.EMA(close['Close'], 7).iloc[-1]
        df['EMA15'] = ta.EMA(close['Close'], 15).iloc[-1]
        df['EMA30'] = ta.EMA(close['Close'], 30).iloc[-1]
        df['EMA90'] = ta.EMA(close['Close'], 90).iloc[-1]
        
        
#         for indicator in ta.get_function_groups()['Pattern Recognition']:
#             df[str(indicator)] = getattr(ta,str(indicator))(df.Open, df.High, df.Low, df.Close)


        # volatility
        
        df_list.append(df)
    
    df_feature_added = pd.concat(df_list)
    
    return df_feature_added



In [10]:
def preprocess_train(df):
    
    # remove columns - Date removed temporarily
    dfc = df.drop(columns=['RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'])
    
    
#     minmax = MinMaxScaler()
    stdsc = StandardScaler()
    ordinal = OrdinalEncoder()

    target = ['Target']
#     minmax_features = ['Date']
    ord_features = ['SecuritiesCode'] 
    scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow',
                      'RSI', 'EMA7', 'EMA15', 'EMA30', 'EMA90'] #+ [c for c in df.columns if c.startswith('CDL')] # pattern recognition features
    
#     date_scaled = minmax.fit_transform(dfc.loc[:,minmax_features])
    date_code_ord = ordinal.fit_transform(dfc.loc[:,ord_features])
    scaled = stdsc.fit_transform(dfc.loc[:,scaled_features])
    
#     display(pd.DataFrame(date_code_ord, columns=ord_features))
#     display(pd.DataFrame(scaled, columns=scaled_features))
    
    
    dfc_scaled = pd.concat([# pd.DataFrame(date_scaled, columns=minmax_features),
                            pd.DataFrame(date_code_ord, columns=ord_features),
                            pd.DataFrame(scaled, columns=scaled_features)], axis=1)

    y = dfc.loc[:,target]
    return dfc_scaled, y, [ordinal, stdsc]
    

X_scaled, y, trained_scalers = preprocess_train(df_added)  # 2021-12-06부터 test 시작이므로 그 전까지만 이용한다.

# X_scaled

In [11]:
def preprocess_inference(df, trained_scalers: list):
    ordinal = trained_scalers[0]
    stdsc = trained_scalers[1]
    
      
    # remove columns - Date removed temporarily
    dfc = df.drop(columns=['RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'])
    
    
    target = ['Target']
    ord_features = ['SecuritiesCode'] 
    scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow',
                      'RSI', 'EMA7', 'EMA15', 'EMA30', 'EMA90'] #+ [c for c in df.columns if c.startswith('CDL')] # pattern recognition features
    
    
    date_code_ord = ordinal.transform(dfc.loc[:,ord_features])
    scaled = stdsc.transform(dfc.loc[:,scaled_features])
    dfc_scaled = pd.concat([pd.DataFrame(date_code_ord, columns=ord_features),
                            pd.DataFrame(scaled, columns=scaled_features)], axis=1)

    
    return dfc_scaled
    

# X_test_scaled = preprocess_train(df_added, trained_scalers)

# X_test_scaled

In [12]:
# base model - lgbm 
lgb = LGBMRegressor().fit(X_scaled, y)



In [13]:
del data, train, supp_data, y, X_scaled
gc.collect()

56

In [14]:
# # local-api
# myapi = local_api('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files')
# env = myapi.make_env()
# iter_test = env.iter_test()
# for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(tqdm(iter_test)):
# #     t0 = time.time()
    
#     # 이전 데이터와 합치고 최근 140일치만 이용한다
#     today = prices.iloc[0]['Date']
#     lastday = str(pd.to_datetime(today) - pd.DateOffset(140))
    
#     if i == 0:
#         close_df = pd.concat([
#             train_with_supp.loc[
#                 (train_with_supp['Date'] > lastday) & (train_with_supp['Date'] < today), 
#                     ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']],
#                 prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']]
#             ]).reset_index(drop=True)
        
#     else:
#         close_df = pd.concat([
#             close_df.loc[
#                 (close_df['Date'] > lastday) & (close_df['Date'] < today), 
#                     ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']],
#                 prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']]
#             ]).reset_index(drop=True)
        

#     feat = add_features_infer(prices, close_df)
#     X = preprocess_inference(feat, trained_scalers)

#     # X, y
#     X['Target'] = lgb.predict(X)
#     X['Rank'] = (X['Target'].rank(method='average', ascending=False)-1).astype(int)
#     sample_prediction['Rank'] = X['Rank'].values
    
#     # check Rank
#     assert sample_prediction["Rank"].notna().all()
#     assert sample_prediction["Rank"].min() == 0
#     assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
    
# #     display(sample_prediction)
#     env.predict(sample_prediction)
# #     print(time.time() - t0)

In [15]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

pd.to_datetime 최소한도로 쓸것. 연산 cost가 너무 큼. 특히 inference 단계에서는 쓰지 않기.

In [16]:
# 2000개 종목 리스트에서 날짜 하나씩 빼고 뒷날짜 추가 구현?


In [17]:
for i, (prices, options, financials, trades, secondary_prices, sample_prediction) in enumerate(iter_test):
#     t0 = time.time()
    
    # 이전 데이터와 합치고 최근 140일치만 이용한다
    today = prices.iloc[0]['Date']
    lastday = str(pd.to_datetime(today) - pd.DateOffset(140))
    
    if i == 0:
        close_df = pd.concat([
            train_with_supp.loc[
                (train_with_supp['Date'] > lastday) & (train_with_supp['Date'] < today), 
                    ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']],
                prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']]
            ]).reset_index(drop=True)
        
    else:
        close_df = pd.concat([
            close_df.loc[
                (close_df['Date'] > lastday) & (close_df['Date'] < today), 
                    ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']],
                prices.loc[:, ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close']]
            ]).reset_index(drop=True)
        

    feat = add_features_infer(prices, close_df)
    X = preprocess_inference(feat, trained_scalers)

    # X, y
    X['Target'] = lgb.predict(X)
    X['Rank'] = (X['Target'].rank(method='first', ascending=False)-1).astype(int)
    sample_prediction['Rank'] = X['Rank'].values
    # check Rank
    assert sample_prediction["Rank"].notna().all()
    assert sample_prediction["Rank"].min() == 0
    assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
#     display(sample_prediction)
    env.predict(sample_prediction)
#     print(time.time() - t0)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
